In [1]:
from PROPERTIES import *

from flask import Flask, request, make_response, Response, session
from flask_wtf.csrf import CSRFProtect, generate_csrf
from flask_sslify import SSLify
from uvicorn import Server, Config
from urllib.parse import quote
from ssl import SSLContext, PROTOCOL_TLS
import ssl
import subprocess
import logging
import random
import string
import uvicorn
import json
import uuid
import binascii
import hmac
import os
import OpenSSL
import datetime
import Cryptodome
import secrets
from asgiref.wsgi import WsgiToAsgi
from cryptography.hazmat.primitives.asymmetric import ec, rsa, padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend, openssl
from cryptography.hazmat.primitives.kdf.concatkdf import ConcatKDFHMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography import x509
from cryptography.x509.oid import NameOID
from Cryptodome.Cipher import PKCS1_OAEP
from Cryptodome.Hash import SHA256
from Cryptodome.PublicKey import RSA

In [2]:
ec_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
ec_public_key = ec_private_key.public_key()

rsa_private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )

subject = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "DE"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, ""),
    x509.NameAttribute(NameOID.LOCALITY_NAME, ""),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, ""),
    x509.NameAttribute(NameOID.COMMON_NAME, "0.0.0.0"),
])

cert = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(subject)
    .public_key(rsa_private_key.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.datetime.utcnow())
    .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))
    .add_extension(
        x509.BasicConstraints(ca=False, path_length=None), critical=True,
    )
    .add_extension(
        x509.SubjectAlternativeName([x509.DNSName("127.0.0.1")]), critical=False,
    )
    .sign(rsa_private_key, hashes.SHA256(), default_backend())
)

private_key_pem = rsa_private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption(),
)

cert_pem = cert.public_bytes(encoding=serialization.Encoding.PEM)

In [3]:
print('ec_public_key hex')
hex = ec_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()

print(hex)

ec_public_key hex
2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d466b77457759484b6f5a497a6a3043415159494b6f5a497a6a304441516344516741457a68626b7751614732356f576c5a666f524f68613639694c395236570a2b6b494f455a675961767052386b705637624173743667544954456532594e6a7553654858346452414b4b593035506430746f57634c543451673d3d0a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a


In [5]:
body = {"digest": "a4e6a9dc2d097df120ae19e26ea9a797ee86a9b50f0bfb9c649c90c549305b34", "key": "2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d466b77457759484b6f5a497a6a3043415159494b6f5a497a6a30444151634451674145466678314669695a6943735a61776d356845734749332b5a6a72677a0a3847683354623641414f30494651564858417a6c337a676b617272626b623658366d444e6f7835532b397977623165763431746256504e4b4b673d3d0a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a"}

sharedSecret_digest_hex = body['digest']
serverPublicKey_hex = body['key']

serialized_ec_public_key_server = bytes.fromhex(serverPublicKey_hex)
ec_public_key_server_on_client = serialization.load_pem_public_key(
    serialized_ec_public_key_server,
    backend=default_backend()
)
# Then computes shared secret and derives key via HKDF
ec_shared_key_on_client = ec_private_key.exchange(
    ec.ECDH(), ec_public_key_server_on_client)
ec_symmetric_key_on_client = ConcatKDFHMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    otherinfo=None,
    backend=default_backend()
).derive(ec_shared_key_on_client)

print('shared secret validation')
print(str(ec_symmetric_key_on_client.hex() == sharedSecret_digest_hex))

cert_hex = binascii.hexlify(cert_pem).decode('utf-8')
# B Symmetrically Encrypts its' RSA Cert with Shared Secret
iv = os.urandom(16)
cipher = Cipher(algorithms.AES(ec_shared_key_on_client),
                modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(bytes.fromhex(cert_hex))
body = {}
body['apiToken'] = "{{apiToken}}"
body['iv'] = iv.hex()
body['key'] = ciphertext.hex()
print(json.dumps(body, indent=4))

shared secret validation
True
{
    "apiToken": "{{apiToken}}",
    "iv": "d40c8dc1b61c62745bfa6a81ca06f46e",
    "key": "4f54ccaf20b780ec8e39249e46b6b88aa0a5d4fedd11258aa92239322d8cbe58839dbb41f584b7f0b0fdb750cdf83b30ed54134b547a4e973ef8078fe71996699c146a6c7bca90766288e6aa06692ad1f7820e5ada97c7740c18df06c930fea9e987ee3507858ef2d8fca1aedd56394ebd147977e82a5ffdc721e5a79ad977a177287a77b94719449047c93ce46fc72ae86ecc15f9b0840209e58de0b9e046b836a62f1edb0dd508b052cbba5a56a806592b570a045397e8b43537748177db4353e4a7eec98293d5b6c05d8c41aa1188c00e83e777dc7048a414045e9d6c4b8cf6a429c9bedd4f507a1ad0636484d60d5d425506b34224a549a6842b6f8174083f8a66d8132e85cd1784dce8a6e331a08c07a98b9397352557bb80fc4e0763bf3d383c1d11caba2550e1f93b3d1661b477fcfd92c361fef950cc363e03b32a9e07f9d451e728c47f007c65b1736682f0f2178ce54b45bd3a1a01b8ba040e9effce65c06a26d77c7c450ec33b2139056ccbab2f6cf38008329cc9af114fd968fa082d75cebcff7fdb412d495cbb2b0882ce67eaee423741d96081045b86daa3fcee09bb1ecb59b7df94ea70f4ed196a99e4c57b198624f5

In [9]:
body = {"key": "d63885e823f9529a2f22f2056b316e4f4fd6dc6cb601732e48eae3243deed11869005f03d8710e2f32392a72c2a3b1537ca544bffa3d653839a5ad98be05a1e8090ab36643a0f2a05e0c38b4f9e5461827a0bdb84ce2a46129aca13b058c958ada2a54d3decee0eafd04dba670e0fa3dcc8ce6cbb440c19b7dd9b9706d4f6e639ce9e5a772e0ddab1ca5275b23bbedfbf3dd4cf8945d43cb3cfd3e9b32195b9fb3ed2cc234dda3f1d87d708cd9eb96cf33365703b398b8fbeffb53c0997bfbafa9aafb5e0bf1f5f151ffc619382b39878fdde592a01eddedc155998280322a967818ed273f22713286a6d444b2f4e3892b3203074b1ae178021e549a01e1b55f64e4f702fd54484c3cc0443738da2ba7f8f42b68760c99040cc5d7098e4e30821f8d4bc953a904f38052e4b212cc3b65c27c8a75cfaf33e8aa1db997eda2fec20faa59b20c0d63a7abf9ff00190a84dd2789495c2687235fceeadb4302c6e6fd16b64db48fae2dd21cb73fcec5ca80cb8ab78a289e97473f47f434812ccced488036072541df149125ddf87cd3bca74dd42a451123ed0fde7d844b6f6fb48ab1f8484ff98c31ad1bde5cc48efb6b36d9a9d44b4ed26845c34c24ae622613f7b478b230d3b16195a3f4850873e4902ed642f46cda32f38d5e172224c29c2e9b614b456b4e85a8c2312c74a4a78ab8855d7a4b852897765f263504efd67f88678ff701273f021472fcbcfb976819282014a2204f52ed79699800c3a11ce7a2aea405c82ced28d230080bdc9e1987c761946b9e4f9a3113ab2a82ef52a18ca285ac2f97761ad3b8a2abfd96c9e4e7c3709b9a1de3faf53ebcb1cd479e4fa17f139ed387060e7d8530d0dd6eb807ea7eabfd33c02c60a170628eac4b906fe79484c45e9722c91138dab59a8531c7bea54c684e71ae5d3fc63a35f96d322e84d75803a274922db792ff6bde25b8d565f99d6474aa100eee951737f9c39d829997e246bd0616ceae34fc5f920f6d02395b0f62027a29ad9e636f4e41aeb944dea0f8d2c8099e76ba6ee9505d70158965118d55675f3407758830a4a65c24bc689eb31ae95f76cb2b69fa0a59e360bb44c2a0274ea6d27efd841ddb5edcb57b25fccb404c52dc221d267f7818d536854431367b3a231dee6888fd8e142cbf7e30f1cd4c006a75b6b9cf87874476a97959d409042168df69f374aadfce3c308275b957c527d355ed4026a124e2500c66034c7eac889189ed6b2a464171b5d9023e7b20817f5727c8393dd14585a6d0a896bff3d83788b557edbf3f9bfdccb679742666eb9b495f5252e4d1199b7ce485b433ad226eff00067dbb01c0b60b85ae4e3ca67d7995f421a1f72282a6c19f80307370c4d4100168fbc1bdbc83daf8f620d2e86b910fc1b906ea995a228921fd1b6b3cff6ecc9ba67a9f8a46c42c6f351603af1b608cb77e643e96026d0e7e6354a2378680c93991c9ce7ef7a2efeaa1bf200b4db2cf9010dade0499d790eadc813e93e8bd30516f6ba7bba84bc2d967b539fae0f49e6546f4f9139bf836c96ef34546c66640a8347fcbf7b1826f4a8215ea9e9528066cf7af3a30f8149db813d08681d038b2486d43eff425958bbbf61255eac4720eb2fca3464b7268105aa6ce29702b9fca6a6e3b6680b6f6383d9e0f414c015bed849e05dd76effb0033ed4781f879b1f895eb0d35fc4b443939ebc8e424b0c2a2e81f50543bbd9625f39a2be865a1012daac720d74de77e98bc198be5f4bafee0872c51a017ae62a3f66f8ef3dc4fe0c73deff50a11cd6d2093e2d00e4eb9a6b5ef50ffc3e77e5bf5e4cf8fdc5412a76138bea07b0b20fb79a0a2e94b8c0d2dd84634e46900e3a619808378b10478aeff8dc43bc31336ff62d41ceb9d514ce1b272c56b55862f49c67ba9535213e0be68f18f1a380b642dee3ed57bcbb506a5bff21ed209350e568e1b8c844be0c842d172dfdef33ebaf68354897ca968098d088a32695295b17bb022aa225931aee1fe6bbaede69f1fe3a399c5c998b3532f203da93c9505515e0664a56228eebc096ee5ee3f1e29db145e2fd900f9889f7c6995458a3957017872939dea880fa99357fa3f648ad9f3426e869dae257ae17fadf23028199c23dcb50202a0a2428dcc7f077b72d07294c9c3b18730fea48e04579df9e0d8be156b510174ed55e09cad48f69de64276a4f234d9753c62f0d80a0be1320aef1d4f9d4f9f0f360888888250316edaba378228b21e05b2d2995547425624940ab5647c28dceebb0b332a01c9cc30eef7dafe35250857c5c0724f7ddef01c2ee0c93eca5d809f795847e7cb7ab19f2ee67415770a2e41bc0cfe44578c72f991eb8792166f2919a7841005c398d25d0864084eb0224a2454aab6948c2fda51942028e36997c39e63e9a3f982eeb812209e52c52c25c1c199ec00bd41b5c1f2a57c2bacd5f5efa8be113939f4535c5c4ea1190f773e0aef0148be5ae9c5ba5342f18aada5cf4a1775972a4b1c6c8f95669bf2e16c838501b4cfd41d", "iv": "5ea1a2ab21d1c84afecadff13af7698b"}

iv_hex = body['iv']
key_hex = body['key']
associated_data=b'seccomAOD'

key_bytes = bytes.fromhex(key_hex)
padding_size = (2 * 32) - 2
#chunk_size = (2048 // 8) - (2 * 32) - 2
chunk_size = 256
encrypted_sym_blocks = [key_bytes[i:i+chunk_size] for i in range(0, len(key_bytes), chunk_size)]
encrypted_asym_blocks = []
plaintext_blocks = []
plaintextDec = b''
# Symmetrical Decryption
decipher_aes = Cipher(algorithms.AES(ec_shared_key_on_client),
                modes.CBC(bytes.fromhex(iv_hex)), backend=default_backend())
decryptor = decipher_aes.decryptor()
for asym_block in encrypted_sym_blocks:
    ciphertext_asym = decryptor.update(asym_block)
    encrypted_asym_blocks.append(ciphertext_asym)
serialized_rsa_private_key = rsa_private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
        )
# Asymmetrical Decryption
loaded_private_key = serialization.load_pem_private_key(
            serialized_rsa_private_key,
            password=None,
            backend=default_backend()
        )
decipher_rsa = PKCS1_OAEP.new(key=loaded_private_key)
for sym_block in encrypted_asym_blocks:
    plaintext_blocks.append(loaded_private_key.decrypt(sym_block, padding=padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=associated_data
    )))
# Reassembly
for plaintext_block in plaintext_blocks:
    plaintextDec = plaintextDec + plaintext_block

print('plaintext')
print(plaintextDec)

channelName = 'fefe'
channelName_bytes = bytes.fromhex(channelName)
cert_peer_on_server = x509.load_pem_x509_certificate(plaintextDec, default_backend())
cert_peer_on_server_bytes = cert_peer_on_server.public_bytes(encoding=serialization.Encoding.PEM)
padding_size = 2 * 32 + 2
nonce = secrets.token_bytes(padding_size)
chunk_size = (2048 // 8) - padding_size
chunks = [channelName_bytes[i:i+chunk_size] for i in range(0, len(channelName_bytes), chunk_size)]
ciphertext_sym_blocks = []
ciphertext_asym_blocks = []
# Asymmetric Encryption of Slized Blocks
cipher_rsa = PKCS1_OAEP.new(key= .public_key(), hashAlgo=hashes.SHA256())
for block in chunks:
    ciphertext_asym_blocks.append(cert_peer_on_server.public_key().encrypt(block, padding=padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=associated_data
    )))
iv_new = os.urandom(16)
# Symmetric Encryption of Slized Blocks
cipher_aes = Cipher(algorithms.AES(ec_shared_key_on_client),
                modes.CBC(iv_new), backend=default_backend())
encryptor = cipher_aes.encryptor()
for asym_block in ciphertext_asym_blocks:
    ciphertext = encryptor.update(asym_block)
    ciphertext_sym_blocks.append(ciphertext)
sym_bytes = b''
# Byte Union
for sym_block in ciphertext_sym_blocks:
    sym_bytes = sym_bytes + sym_block
body = {}
body['apiToken'] = "{{apiToken}}"
body['userToken'] = "{{userToken}}"
body['iv'] = iv_new.hex()
body['channelName'] = sym_bytes.hex()
print(json.dumps(body, indent=4))

ValueError: Encryption/decryption failed.